# AutoModelizer
---

La idea de este proyecto es encontrar el mejor modelo de CNN que se adapte al dataset correspondiente, para ello usando algoritmos evolutivos. Este tipo de soluciones se conocen como neuroevoluciones

A continuación un ejemplo básico de como funcionan este tipo de algoritmos

In [1]:
import numpy as np
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


def fitness(x):
    return x ** 2

population_size = 10
population = np.random.uniform(-10, 10, population_size)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def select_parent_tournament(population, scores, k=3):
    selection_ix = np.random.randint(len(population), size=k)
    selected = population[selection_ix]
    ix = np.argmax(scores[selection_ix])
    return selected[ix]

def crossover(p1, p2):
    child = (p1 + p2) / 2
    return child

def mutate(x):
    mutation_chance = 0.1
    if np.random.rand() < mutation_chance:
        x += np.random.uniform(-1, 1)
    return x

In [3]:
n_generations = 200

for generation in range(n_generations):
    scores = np.array([fitness(x) for x in population])
    new_population = []
    for _ in range(population_size):
        parent1 = select_parent_tournament(population, scores)
        parent2 = select_parent_tournament(population, scores)
        child = crossover(parent1, parent2)
        child = mutate(child)
        new_population.append(child)
    population = np.array(new_population)
    best_score = np.max(scores)
    print(f"Generación {generation}, x = {child} Mejor puntuación {best_score}")

best_solution = population[np.argmax(scores)]
print(f"Mejor solución: x = {best_solution}, f(x) = {fitness(best_solution)}")


Generación 0, x = 7.5494467764944595 Mejor puntuación 76.69341927871703
Generación 1, x = 7.74815554317817 Mejor puntuación 56.99414663112258
Generación 2, x = 7.5494467764944595 Mejor puntuación 60.033914321282595
Generación 3, x = 7.827564799677173 Mejor puntuación 65.70209402480582
Generación 4, x = 7.8772419913481 Mejor puntuación 63.69287515253533
Generación 5, x = 7.929011802115674 Mejor puntuación 62.869228158089655
Generación 6, x = 7.909859700464915 Mejor puntuación 62.869228158089655
Generación 7, x = 7.922802153156809 Mejor puntuación 62.869228158089655
Generación 8, x = 7.922802153156809 Mejor puntuación 62.869228158089655
Generación 9, x = 7.922802153156809 Mejor puntuación 62.869228158089655
Generación 10, x = 8.130587145594077 Mejor puntuación 69.42493174430649
Generación 11, x = 8.129810939474218 Mejor puntuación 78.51624608260641
Generación 12, x = 8.67745997051053 Mejor puntuación 78.51624608260641
Generación 13, x = 9.102225837593931 Mejor puntuación 89.7890145057374

Preparamos el dataset de prueba (MNIST)
---

---

In [4]:
# DATOS DEL DATASET DE PRUEBA

num_channels = 1
px_h = 28
px_w = 28
batch_size = 64
num_classes = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preparar los DataLoader para los conjuntos de entrenamiento y validación
transform = transforms.Compose([
    transforms.Resize((px_h, px_w)),
    transforms.Grayscale(num_output_channels=num_channels),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
val_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)


Preparamos el dataset de prueba (CIFAR100)
---

---

In [5]:

# DATOS DEL DATASET DE PRUEBA

num_channels = 3
px_h = 32
px_w = 32
batch_size = 32
num_classes = 100

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preparar los DataLoader para los conjuntos de entrenamiento y validación
transform = transforms.Compose([
    transforms.Resize((px_h, px_w)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
val_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)


'\n# DATOS DEL DATASET DE PRUEBA\n\nnum_channels = 3\npx_h = 32\npx_w = 32\nbatch_size = 32\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n\n# Preparar los DataLoader para los conjuntos de entrenamiento y validación\ntransform = transforms.Compose([\n    transforms.Resize((px_h, px_w)),\n    transforms.ToTensor(),\n    transforms.Normalize((0.5,), (0.5,))\n])\n\ntrain_dataset = datasets.CIFAR100(root=\'./data\', train=True, download=True, transform=transform)\nval_dataset = datasets.CIFAR100(root=\'./data\', train=False, download=True, transform=transform)\n\ntrain_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)\nval_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)\n'

## GENERACIÓN DE LA RED EN BASE A VECTORES


---

comprobamos si tenemos acceso a la GPU


In [6]:
if torch.cuda.is_available():
    print("CUDA (GPU) está disponible en tu sistema.")
else:
    print("CUDA (GPU) no está disponible en tu sistema.")

CUDA (GPU) está disponible en tu sistema.


Generamos un par de arquitecturas de prueba

Construimos una función que sea capaz de crear modelos en base a vectores que representen la arquitectura de la red.
De este modo el algorimo evolutivo puede ir adaptando y cambiando la red fácilmente

In [7]:
import torch

def build_cnn_from_individual(individual):
    layers = []
    num_layers = individual['num_conv_layers']
    out_channels_previous_layer = num_channels # Imagen de entrada en escala de grises (1 canal para MNIST)

    for i in range(num_layers):
        out_channels = individual['filters'][i]
        kernel_size = individual['filter_sizes'][i]
        
        conv_layer = nn.Conv2d(out_channels_previous_layer, out_channels, kernel_size=kernel_size, padding=1)
        layers.append(conv_layer)
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))

        out_channels_previous_layer = out_channels

    # Temporalmente crear un modelo para calcular el tamaño de salida de las capas convolucionales
    temp_model = nn.Sequential(*layers)

    # Calcular el tamaño de salida usando un tensor dummy
    dummy_input = torch.zeros(1, num_channels, px_h, px_w)  # Tamaño de entrada para MNIST
    output_size = temp_model(dummy_input).view(-1).shape[0]

    # Ahora, sabiendo el tamaño de salida, podemos definir las capas lineales correctamente
    layers.append(nn.Flatten())
    layers.append(nn.Linear(output_size, num_classes))  # Salida de 10 clases para MNIST
    return nn.Sequential(*layers)


Definimos la población inicial

In [8]:
import random
from tqdm import tqdm

def generate_individual(min_conv_layers, max_conv_layers, min_filters, max_filters, filter_sizes, lr_min, lr_max):
    individual = {
        'num_conv_layers': random.randint(min_conv_layers, max_conv_layers),
        'filters': [],
        'filter_sizes': [],
        'learning_rate': random.uniform(lr_min, lr_max),
    }

    for _ in range(individual['num_conv_layers']):
        individual['filters'].append(random.randint(min_filters, max_filters))
        individual['filter_sizes'].append(random.choice(filter_sizes))
    
    # Agrega más parámetros según sea necesario, como capas completamente conectadas, etc.

    return individual

def initialize_population(pop_size, min_conv_layers, max_conv_layers, min_filters, max_filters, filter_sizes, lr_min, lr_max):
    return [generate_individual(min_conv_layers, max_conv_layers, min_filters, max_filters, filter_sizes, lr_min, lr_max) for _ in range(pop_size)]


### PARAMETROS PARA POSIBLES ARQUITECTURAS DE RED
---

In [9]:
## AJUSTAR SEGÚN VAYA NECESITANDO Y TIEMPO 

population_size = 10
min_conv_layers = 1
max_conv_layers = 3
min_filters = 16
max_filters = 128
filter_sizes = [3, 5]
lr_min = 0.0001
lr_max = 0.01

In [10]:


population = initialize_population(population_size, min_conv_layers, max_conv_layers, min_filters, max_filters, filter_sizes, lr_min, lr_max)

epochs = 10

population


[{'num_conv_layers': 2,
  'filters': [100, 109],
  'filter_sizes': [3, 3],
  'learning_rate': 0.004766873762543886},
 {'num_conv_layers': 2,
  'filters': [24, 75],
  'filter_sizes': [5, 5],
  'learning_rate': 0.003982881181242181},
 {'num_conv_layers': 2,
  'filters': [48, 78],
  'filter_sizes': [3, 3],
  'learning_rate': 0.007001647934110196},
 {'num_conv_layers': 3,
  'filters': [50, 109, 74],
  'filter_sizes': [3, 3, 3],
  'learning_rate': 0.003755493959258847},
 {'num_conv_layers': 1,
  'filters': [99],
  'filter_sizes': [5],
  'learning_rate': 0.0037280778791694328},
 {'num_conv_layers': 1,
  'filters': [118],
  'filter_sizes': [3],
  'learning_rate': 0.003801014160816255},
 {'num_conv_layers': 2,
  'filters': [80, 103],
  'filter_sizes': [3, 5],
  'learning_rate': 0.002901995463893161},
 {'num_conv_layers': 1,
  'filters': [67],
  'filter_sizes': [3],
  'learning_rate': 0.005202401067199825},
 {'num_conv_layers': 1,
  'filters': [33],
  'filter_sizes': [3],
  'learning_rate': 0.0

## ENTRENAMIENTO Y EVALUACIÓN DE LOS MODELOS
---

In [11]:
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

def evaluate_individual(individual, train_loader, val_loader, device='cuda', epochs=5):
    # Construir el modelo basado en el individuo
    model = build_cnn_from_individual(individual).to(device)
    
    # Definir el optimizador y la función de pérdida
    optimizer = torch.optim.Adam(model.parameters(), lr=individual['learning_rate'])
    criterion = nn.CrossEntropyLoss()

    # Entrenamiento
    for epoch in range(epochs):
        model.train()
        progress_bar = tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{epochs}', unit='batch')
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

            # Actualizar la barra de progreso con la última información de pérdida
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item())})
            progress_bar.update()  # Forzar la actualización de la barra de progreso
            
        progress_bar.close()  # Cerrar la barra de progreso al final de cada época

    # Evaluación
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in val_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = correct / total
    return accuracy  # Esta es la "aptitud" del individuo


## MUTACIONES Y CRUCES
---

In [12]:
def mutate_individual(individual):
    """
    Mutar un individuo cambiando aleatoriamente sus hiperparámetros.
    """
    # Define los rangos de mutación para cada hiperparámetro
    mutation_rate = 0.1  # Probabilidad de mutar cada característica
    lr_range = (0.0001, 0.01)
    conv_range = (1, 5)

    if random.random() < mutation_rate:
        # Mutar la tasa de aprendizaje
        individual['learning_rate'] = random.uniform(*lr_range)

    for i in range(len(individual['num_conv_layers'])):
        if random.random() < mutation_rate:
            # Mutar el número de capas
            individual['num_conv_layers'][i] = random.randint(*conv_range)

    return individual

Y el cruce entre individuos

In [13]:
def crossover(parent1, parent2):
    """
    Realiza un cruce uniforme entre dos individuos.
    
    Args:
        parent1 (dict): El primer individuo padre.
        parent2 (dict): El segundo individuo padre.
    
    Returns:
        dict: Un nuevo individuo hijo.
    """
    child = {}
    
    for key in parent1:
        if random.random() < 0.5:
            child[key] = parent1[key]
        else:
            child[key] = parent2[key]
            
    return child

In [14]:
def evaluate_population(population, train_loader, val_loader, device):
    fitness_scores = []
    
    for individual in population:
        print(individual)
        fitness = evaluate_individual(individual, train_loader, val_loader, device)
        fitness_scores.append(fitness)
    
    return fitness_scores




Recopilamos lo que hemos realizado, hemos creado las posibles mutaciones sobre las arquitecturas, los posibles cruces, la evaluación de los modelos.


Queda realizar:
- Selección de reproducción: por torneo en principio para también  puede ser por torneo o ruleta
- Creación de la nueva generación usando las funciones de mutación y cruces
- Criterios de parada
- Registro de análisis

## ALGORITMO EVOLUTIVO
--- 


In [15]:
# Asumiendo que 'population' ya ha sido inicializada
fitness_scores = evaluate_population(population, train_loader, val_loader, device)

# Opcional: Almacenar los individuos y sus puntuaciones en una lista de tuplas y ordenarlos
population_with_scores = list(zip(population, fitness_scores))
population_with_scores.sort(key=lambda x: x[1], reverse=True)  # Ordena de mayor a menor aptitud

# Imprime los resultados
for i, (individual, score) in enumerate(population_with_scores):
    print(f"Individuo {individual}: Aptitud = {score}")

{'num_conv_layers': 2, 'filters': [100, 109], 'filter_sizes': [3, 3], 'learning_rate': 0.004766873762543886}


Epoch 1/5:  75%|███████▍  | 702/938 [00:13<00:04, 50.08batch/s, training_loss=0.046]

KeyboardInterrupt: 

## RESULTADOS DE LAS ARQUITECTURAS
---

In [ ]:
for i, (individual, score) in enumerate(population_with_scores):
    print(f"Individuo {individual}: Aptitud = {score}")